In [1]:
import pickle
import tqdm
import collections 
import re
import numpy as np

In [29]:
single_modal_dict_prefix = '/gpfs/data/geraslab/Ashen/multimodal/single_modality_dictionary/01102022'

with open(os.path.join(single_modal_dict_prefix, 'ffdm_screening.pkl') , "rb") as f:
    ffdm_screening_data = pickle.load(f)

In [4]:
file = "/gpfs/data/geraslab/Nan/StudiesScreeningMammo/1202_reader_study/1208_data_list_and_tracking_table.pkl"
with open(file, "rb") as f:
    read_study_data = pickle.load(f)

In [24]:
ffdm_exam_list = [e for exams in read_study_data[0][1:] for e in exams]

In [34]:
def get_ffdm_info_list(ffdm_exam, diagnostic, lateral):
    
    img_prefix = '/gpfs/data/geraslab/jp4989/data/2021.07.16.combined_ffdm_cropped'
    if diagnostic:
        img_prefix = f"{img_prefix}_diagnostic"
    accession_number = ffdm_exam['accession_number']
    
    ffdm_info_list = []
    for view in ffdm_exam['original_image_size'].keys():
        if view.startswith(lateral[0].upper()):
            images = ffdm_exam[view]
            for index, filename in enumerate(images):
                info = {'img_prefix': img_prefix, 
                        'accession_number': accession_number, 
                        'index': index,
                        'view': view,
                        'filename': filename,
                        'horizontal_flip': ffdm_exam['horizontal_flip'][view][index] if diagnostic \
                            else ffdm_exam['horizontal_flip'], 
                        'best_center': ffdm_exam['best_center'][view][index],
                        'crop_method': 'no_crop' if diagnostic else 'best_center'
                }
                ffdm_info_list.append(info)
    return ffdm_info_list
    
def generate_list(exam, ffdm_data=ffdm_screening_data):
    
    breasts = []
    for lateral in ['left', 'right']:
        ffdms = get_ffdm_info_list(ffdm_data[exam['accession_number']], False, lateral)

        for ffdm in ffdms:
            breast_instance = {'accession_number': exam['accession_number'],
                                'lateral': lateral,
                                'mrn': exam['patient_ID'], 
                                'study_date': exam['study_date'],
                                'benign': exam['cancer_label'][f"{lateral}_benign"], 
                                'malignant': exam['cancer_label'][f"{lateral}_malignant"],
                                'ffdm': ffdm}
            breasts.append(breast_instance)
            
    return breasts

In [37]:
ffdm_image_list = [img for exam in ffdm_exam_list for img in generate_list(exam)]

3110

In [41]:
prefix = '/gpfs/data/geraslab/Nan/data/breast_mml_datalists/20220111/breasts_lists'
savepath = 'nyu_readerstudy_ffdm'
phase = 'val'

# os.mkdir(os.path.join(prefix, savepath))
file = os.path.join(prefix, savepath, phase)

print(phase, "#pairs:", len(ffdm_image_list), '\n\t saved at: ', savepath)

with open(file, 'wb') as f:
    pickle.dump(ffdm_image_list, f)

val #pairs: 3110 
	 saved at:  nyu_readerstudy_ffdm


In [1]:
# ffdm_image_list[0]